<a href="https://colab.research.google.com/github/Venus2Mice/Venus2Mice/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [1]:
!pip install -U -q google-generativeai

In [2]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [6]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [7]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [8]:
model = 'gemini-1.5-pro-002' # @param {isTemplate: true}
contents_b64 = '...' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [ ]:
gais_contents

In [ ]:
generation_config

In [ ]:
safety_settings

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [9]:
# @title `show_file` function
drive = None
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [ ]:
for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [10]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)

drive = None
def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [11]:
contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Here is the coverted `Content`s:

In [12]:
contents

[{'role': 'user',
  'parts': [{'text': 'dịch văn bản sau sang tiếng việt : 展颜一笑，阿婉便是转身离开了这亭子，只留柔小仪一人在亭中不知是在想些什么。\\n\\n\u3000\u3000阿婉回到了凉霄阁不久，李得闲便是过来请了她前去万和楼，本以为是皇上忙完了政事儿，闲暇时有空召见自己，没想到进了万和楼的书房内，除了皇上外，却还瞧见一个藏青色长衫，身材清瘦，与自己还有几分相像的男子立在其中，一时间惊喜得也顾不上礼仪，便是脱口而出：\\n\\n\u3000\u3000“大哥哥，你怎的在这儿！”\\n\\n\u3000\u3000作者有话要说：\\n\\n\u3000\u3000作者君已挂请烧纸，谢谢。\\n\\n\u3000\u3000☆、第74章 事发\\n\\n\u3000\u3000书房内所立之人的确是阿婉的大哥,温洵。\\n\\n\u3000\u3000历经一年的官场历练，若说之前的温洵是行事有度，那么现在看起来更显稳重成熟,面上也褪去了青涩的少年模样,取而代之的是眼底不可磨灭的一股坚毅，不改的是那清瘦的身量立于书房内，仿佛如同笔直的竹子一般，还是透出坚毅不屈、从容不迫的君子风骨。\\n\\n\u3000\u3000一听到熟悉却是长久未听到的声音,温洵便知是妹妹来了,也有将近一年未见妹妹,温洵心中虽是记挂得很,但依旧不忘君臣礼仪,极力不去看阿婉,对着皇上屈身拱手道，“婉昭仪娘娘有礼，臣以为还是先与皇上行礼较妥。”\\n\\n\u3000\u3000阿婉一时间也是惊喜过度，经兄长提醒才记起这茬儿，“臣妾给皇上请安，皇上万福金安，臣妾许久未见兄长，一时失态，还请皇上恕罪。”\\n\\n\u3000\u3000上首的皇上打量这兄妹俩，长相倒是几分相似，可一个性子沉稳，颇有谋略，另一个则是性子跳脱，行事出人意料，这般南辕北辙的性子竟是一母同胞的兄妹，倒还真是有趣。\\n\\n\u3000\u3000“温爱卿无需多礼，婉昭仪起吧，赐座。”有旁人在时，皇上是甚少叫阿婉的名字的，即便在场的是阿婉的兄长，皇上也并不以昵称称呼阿婉，可单是赐座这一举动，温洵对于妹妹在宫里的日子过得如何还是有了些了解。\\n\\n\u3000\u3000“不知皇上唤臣妾来有何事？”坐下后阿婉便是率先开了口，并不在意哥哥不赞同的

## Call `generate_content`

In [13]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [14]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

ValueError: Invalid operation: The `response.parts` quick accessor requires a single candidate, but but `response.candidates` is empty.

In [15]:
response.candidates

[]

## Or Create a chat

In [16]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [17]:
model_turn = contents[-1].get("role", None) == "user"

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])

    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

In [18]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [ ]:
# response = chat.send_message("Interesting, tell me more.")
# display(Markdown(response.text))